In [7]:
pip install spark requests

Note: you may need to restart the kernel to use updated packages.


In [168]:
import pyspark
import requests
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, regexp_replace, avg, mean
from pyspark.sql.types import FloatType
from os.path import exists

In [83]:
spark = SparkSession.builder.appName('Exercise').getOrCreate()
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'

In [84]:
response = requests.get(url)

In [85]:
if (response.status_code == 200):
    spliter = url.split('/')
    filename = spliter[len(spliter) - 1]
    if not exists(filename):
        if open(filename, "wb").write(response.content) == False:
            print("Download failed")
        else:
            print("Download succeced")

In [111]:
df = spark.read.csv(filename, sep=r'\t', header=True, inferSchema=True)

In [99]:
df.show()

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
|       5|       1| Chips and Guacamole|           

In [88]:
df.count()

4622

In [89]:
len(df.columns)

5

In [90]:
df.columns

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

In [91]:
df.groupby('item_name').count().orderBy(col("count").desc()).show(1)


+------------+-----+
|   item_name|count|
+------------+-----+
|Chicken Bowl|  726|
+------------+-----+
only showing top 1 row



In [92]:
df.filter(df['choice_description'] != 'NULL').groupby('choice_description').count().orderBy(col("count").desc()).show(1)

+------------------+-----+
|choice_description|count|
+------------------+-----+
|       [Diet Coke]|  134|
+------------------+-----+
only showing top 1 row



In [93]:
df.select(sum(df.quantity)).show()

+-------------+
|sum(quantity)|
+-------------+
|         4972|
+-------------+



In [94]:
df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)



In [133]:
df = df.withColumn('item_price', regexp_replace(col('item_price'), "[$]", "").alias("replaced"))

In [137]:
df = df.withColumn("item_price", df.item_price.cast(FloatType()))

In [138]:
df.select(sum(df.item_price)).show()

+------------------+
|   sum(item_price)|
+------------------+
|34500.159754514694|
+------------------+



In [142]:
df.orderBy(col("order_id").desc()).show(1)

+--------+--------+------------------+--------------------+----------+
|order_id|quantity|         item_name|  choice_description|item_price|
+--------+--------+------------------+--------------------+----------+
|    1834|       1|Chicken Salad Bowl|[Fresh Tomato Sal...|     11.25|
+--------+--------+------------------+--------------------+----------+
only showing top 1 row



In [156]:
df = df
tmp = df.groupby("order_id").sum("item_price")

In [169]:
tmp.select(mean(tmp[1])).show()

+--------------------+
|avg(sum(item_price))|
+--------------------+
|   18.81142843757617|
+--------------------+



In [167]:
df.select("item_name").distinct().count()

50

In [175]:
df.groupby('order_id').sum().show()

+--------+-------------+-------------+------------------+
|order_id|sum(order_id)|sum(quantity)|   sum(item_price)|
+--------+-------------+-------------+------------------+
|     148|          592|            4|  30.9399995803833|
|     463|          926|            2|10.679999828338623|
|     471|         2355|            5|24.149999618530273|
|     496|         2480|            5|17.549999952316284|
|     833|         1666|            2|             12.75|
|    1088|         2176|            2|23.140000343322754|
|    1238|         2476|            2|13.169999599456787|
|    1342|         4026|            3|12.150000095367432|
|    1580|         4740|            3| 21.46999955177307|
|    1591|         1591|            2|              18.5|
|    1645|         4935|            3|              12.0|
|    1829|         5487|            3|             24.25|
|     243|          243|            2|              22.5|
|     392|         1176|            3|11.269999861717224|
|     540|    